---
# Diagnosis of mechanical rotor faults in drones using functional gaussian mixture classifier
B. Bartoszewski,
K. Jarzyna
---

---

### Problem formulation [0-4 pts]:
  - is the problem clearly stated [1 pt]
  - what is the point of creating model, are potential use cases defined [1 pt]
  - where do data comes from, what does it containt [1 pt]
  - DAG has been drawn [1 pt]
  - confoundings (pipe, fork, collider) were described [1 pt]

#### *Detection of Drone Propeller Damage*

<style>
.justify {
    text-align: justify;
}
</style>

<div class="justify">
A program has been developed for detecting damage to drone propellers, based on data from the accelerometer and gyroscope. The system analyzes changes in the drone's vibrations and movements, allowing for early detection of anomalies indicating mechanical damage to the propellers. This enables quick corrective actions, minimizing the risk of further damage, and improving the safety and reliability of drone operations.
</div>



#### *Potential use*
<style>
.justify {
    text-align: justify;
}
</style>

<div class="justify">
By analyzing vibration patterns, the system can quickly identify anomalies that may suggest damage or wear of components. When unusual vibrations are detected, the system immediately alerts the operator, enabling prompt actions such as controlled landing for inspection and potential replacement of damaged parts.

Using this program can reduce the risk of multirotor flights and decrease the operational cost of drones by mitigating the potential destruction of the entire drone or surrounding infrastructure.
</div>



#### *Collecting Data*
<style>
.justify {
    text-align: justify;
}
</style>

<div class="justify">
The data was collected by our own. To gather data, flights were conducted using an eight-rotor drone. Initially, flights were carried out with all propellers in good condition, followed by series of flights with one damaged propeller measuring 1.5 cm in length. Subsequently, the damaged propeller was replaced with one measuring 3 cm in length, and the series of tests continued.
</div>

<style>
.justify {
    text-align: justify;
}
.center {
    display: block;
    margin-left: auto;
    margin-right: auto;
    width: 50%;
}
</style>

<div class="justify">
</div>
<p style="text-align: center;">A healthy drone used for collecting data.</p>
<img src="photos/IMG_3984.JPEG" alt="Photo of drone to collect data" class="center"/>

<p style="text-align: center;">One propeller cut 1.5 cm</p>
<img src="photos/1_5cm.jpg" alt="Photo of drone to collect data" class="center"/>

<p style="text-align: center;">One propeller cut 3 cm</p>
<img src="photos/3cm.jpg" alt="Photo of drone to collect data" class="center"/>





<style>
.justify {
    text-align: justify;
}
</style>

<div class="justify">

The system used in the drone is [Ardupilot](https://ardupilot.org/). It is an open-source autopilot designed for controlling, among other things, unmanned aerial vehicles.

One of the important features of ArduPilot is the ability to record flight data onto an external SD card. The capability of collecting flight logs was utilized, where the appropriate [logging parameters](https://ardupilot.org/copter/docs/parameters.html#log-bitmask) needed to be selected before the flight to gather data from the flights.

</div>

<p style="text-align: center;">Set <a href="https://ardupilot.org/copter/docs/parameters.html#log-bitmask" target="_blank">logging parameters</a></p>
<img src="https://ardupilot.org/copter/_images/mp_dataflash_log_bitmask.png" alt="Opis obrazu" width="1000"  class="center"/>

---
### Data preprocessing   [0-2 pts]:
- is preprocessing step clearly described [1 pt]
- reasoning and types of actions taken on the dataset have been described [1 pt]



#### *Data preprocessing*

<style>
.justify {
    text-align: justify;
}
</style>

<div class="justify">
After collecting data, files from the `logs_from_flights` folder were replayed in the [UAV Log Viewer](https://plot.ardupilot.org/) program to export data from the gyroscope and accelerometer. This program was used because it allowed for a clear visualization of the drone's flight trajectory and ensured that the exported data corresponded to the appropriate flights with the relevant damages.
</div>



<style>
.justify {
    text-align: justify;
}
.center {
    display: block;
    margin-left: auto;
    margin-right: auto;
    width: 50%;
}
</style>

<div class="justify">
</div>
<p style="text-align: center;">Data visualization in <a href="https://plot.ardupilot.org/" target="_blank">UAV Log Viewer</a></p>
<img src="photos/uav_logs.png" alt="Opis obrazu" width="27530"  class="center"/>


<style>
.justify {
    text-align: justify;
}
</style>

<div class="justify">
In file preprocessing.ipynb was made preprocessing from data whitch was generated in UAV Log Viewer. We saved thee types of data:

- accelerometer data in the time domain
- gyroscope data in the frequency domain
- gyroscope data in the frequency domain with accumulation

The data was subjected to the removal of unit jumps that occur during takeoff and landing, which could reduce the model's effectiveness. Additionally, gyroscope data was subjected to Fourier transformation to examine how vibrations behave at different frequencies. Before exporting the data to the model, the flight data was divided into segments every 1.5 seconds. This aims to match the input data to the models, where the first and second models have a sliding window, and the third model aggregates the data.
</div>



---
### Model [0-4 pts]:
- are models sufficiently described (what are formulas, what are parameters, what data are required ) [1 pt] 
- are difference between two models explained [1 pt]  
- is the difference in the models justified (e.g. does adding aditional parameter makes sense? ) [1 pt]  
- are two different models specified [1 pt]

We decided to use Bayesian Mixture Model as our binary classifier. We assume that a given measurement, $y$, represented using B-spline basis can be drawn from one of $M$ data generating processes (which represents our fault classes), each described by unique set of parameters. To create this model, we need to build the likelihood function and then use it to develop the posterior distribution. 

Data generating process is given by

$$ y \sim \mathcal{Normal}(\mu, \sigma) $$
$$ \mu = \sum_{m=1}^M \beta_m \phi_m(t) $$

where $y$ is sampled signal with uncertanity of normal distribution given by $\sigma$. Functions $\phi_m(t)$ are B-splines on assigned M knot grid. $\mu$ represents transformed parameters of the model, as it corresponds to the mean of fitted distribution of each class. $\beta_m$ are terms regulating each spline values, all are descirbed by normal distribution

$$ \beta_m \sim \mathcal{Normal}(\mu_b,\sigma_b) $$

Our two model differs in number of used splines to represent signals, the more we use the better we can approximate signal however we are also greatly increasing computational time and risk overfitting. First model uses 15 splines and second 57. We decided on this numbers because initial experiments yielded promising results with low number of basis functions, however increasing them one by one was slowly improving correct classifications.


---
### Priors [0-4 pts]: Kacper
- Have prior predictive checks been done for parameters (are parameters simulated from priors make sense) [1 pt]
- How prior parameters were selected [1 pt]
- Have prior predictive checks been done for measurements (are measurements simulated from priors make sense) [1 pt]
- Is it explained why particular priors for parameters were selected [1 pt]


We have chosen normal distribution for our beta parameters since 

---
### Posterior analysis (model 1) [0-4 pts]: KAcper
- have parameter marginal disrtibutions been analyzed (histograms of individual parametes plus summaries, are they diffuse or concentrated, what can we say about values) [1 pt]
- were there any issues with the sampling? if there were what kind of ideas for mitigation were used [1 pt] 
- are the data consistent with posterior predictive samples and is it sufficiently commented (if they are not then is the justification provided)
- are the samples from posterior predictive distribution analyzed [1 pt] 

---
### Posterior analysis (model 2) [0-4 pts]:
- have parameter marginal disrtibutions been analyzed (histograms of individual parametes plus summaries, are they diffuse or concentrated, what can we say about values) [1 pt]
- were there any issues with the sampling? if there were what kind of ideas for mitigation were used [1 pt]
- are the samples from posterior predictive distribution analyzed [1 pt]
- are the data consistent with posterior predictive samples and is it sufficiently commented (if they are not then is the justification provided)

---
### Model comparison [0-4 pts]: Bartosz, Kacper
- Have models been compared using information criteria [1 pt]
- Have result for WAIC been discussed (is there a clear winner, or is there an overlap, were there any warnings) [1 pt]
- Have result for PSIS-LOO been discussed (is there a clear winner, or is there an overlap, were there any warnings) [1 pt]
- Was the model comparison discussed? Do authors agree with information criteria? Why in your opinion one model better than another [1 pt]
